In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gseapy as gp

In [2]:
deg = pd.read_csv("/mnt/c/Users/ouyangkang/Downloads/all_diff_peak_se.csv")

In [3]:
deg.head()

,Unnamed: 0,seqnames,idx,start,end,Log2FC,FDR,MeanDiff,peakName,se_id,nearestGene
0,1,chr9,12191,61168938,61169438,1.223151,0.000340,0.593956,chr9:61168938-61169438,SE_12_025400461,I730028E13Rik
1,2,chr8,14433,89404193,89404693,1.644044,0.001825,0.551949,chr8:89404193-89404693,SE_12_025400088,Sall1
2,3,chr8,14426,89392632,89393132,1.035705,0.010923,0.552092,chr8:89392632-89393132,SE_12_025400088,Sall1
3,4,chr8,14431,89396602,89397102,1.944952,0.025567,0.374053,chr8:89396602-89397102,SE_12_025400088,Sall1
4,5,chr13,6280,43778204,43778704,1.262150,0.084380,0.318489,chr13:43778204-43778704,SE_12_045400188,Cd83


In [8]:
mgl_df = deg[deg['Log2FC'] >= 0].sort_values(by='Log2FC', ascending=False)
mac_df = deg[deg['Log2FC'] <= 0].sort_values(by='Log2FC')
mac_df.head()

,Unnamed: 0,seqnames,idx,start,end,Log2FC,FDR,MeanDiff,peakName,se_id,nearestGene
2895,2896,chr1,11632,84882943,84883443,-5.439819,0.059347,-0.255457,chr1:84882943-84883443,SE_12_045700798,Trip12
2894,2895,chr16,2591,12032347,12032847,-5.317884,0.045216,-0.234265,chr16:12032347-12032847,SE_12_045700430,Shisa9
2920,2921,chr4,25638,143278255,143278755,-5.190113,0.342052,-0.213899,chr4:143278255-143278755,SE_12_045400191,Pdpn
2897,2898,chr8,19794,117074485,117074985,-5.184113,0.115264,-0.212987,chr8:117074485-117074985,SE_12_045600282,Pkd1l2
2918,2919,chr11,26801,109671780,109672280,-5.096357,0.342052,-0.200062,chr11:109671780-109672280,SE_12_045500630,Prkar1a


In [56]:
top_activated_mgl_genes = []
top_mac_genes = []

mgl_genes = mgl_df['nearestGene'].tolist()
top_activated_mgl_genes += list(set(mgl_genes[:200]))

if len(top_activated_mgl_genes) != 200:
    for i in range(10000):
        top_activated_mgl_genes += mgl_genes[i+201]
        top_activated_mgl_genes = list(set(top_activated_mgl_genes))
        if len(top_activated_mgl_genes) >= 200:
            break

mac_genes = mac_df['nearestGene'].tolist()
top_mac_genes += list(set(mac_genes[:200]))

if len(top_mac_genes) != 200:
    for i in range(1000):
        top_mac_genes += mac_genes[i+201]
        top_mac_genes = list(set(top_mac_genes))
        if len(top_mac_genes) >= 200:
            break


In [59]:
top_mac_genes.pop()

'Pygb'

In [61]:
gp.get_library_name(organism='Mouse')

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'Azimuth_2023',
 'Azimuth_Cell_Types_2021',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'COVID-19_Related_Gene_Sets',
 'COVID-19_Related_Gene_Sets_2021',
 'Cancer_Cell_Line_Encyclopedia',
 'CellMarker_2024',
 'CellMarker_Augmented_2021',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'ChEA_2022',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DGIdb_Drug_Targets_2024',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_CRISPR_GeneDependency_CellLines_2023',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_Cell

In [62]:
def enrichment(gene_list, file_name_prefix):
    # enrichment analysis
    enr = gp.enrich(gene_list,
                    gene_sets="GO_Biological_Process_2023",
                    outdir=None)
    # filtering p value is more than 0.05
    result = enr.res2d[enr.res2d['P-value'] < 0.05]
    # save result
    result.to_csv(f"/mnt/c/Users/ouyangkang/Desktop/{file_name_prefix}_result.csv", index=None)

    return result

In [69]:
df = pd.DataFrame({
    "MAC": top_mac_genes,
    "activated_MGL": top_activated_mgl_genes
})

df.to_csv('/mnt/c/Users/ouyangkang/Desktop/diff_marker.csv', index=None, encoding='utf-8')

In [64]:
mgl_result = enrichment(top_activated_mgl_genes[:100], 'activated_mgl_diff')
mac_result = enrichment(top_mac_genes[:100], 'mac_diff')

In [ ]:
select_mgl = mgl_result.iloc[[0, 1, 2 ,5, 8, 14, 15, 18, 19, 22]]
select_mgl["Lable"] = "activated MGL"
select_mac = mac_result.iloc[[0, 1, 2, 3, 5, 6, 8, 16, 24, 39]]
select_mac["Lable"] = "MAC"